# Context-FID Score Presentation
## Necessary packages and functions call

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [1]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading

Load original dataset and preprocess the loaded data.

In [2]:
iterations = 5
dataset_name = 'energy'
seq_length = 24
# ori_data = np.load('../toy_exp/samples/energy_ground_truth_32_train.npy')
ori_data = np.load(f'../energy_results/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../energy_results/ddpm_fake_energy_0_to1.npy')

## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [13]:
context_fid_score = []

for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.23162601425381216 

Iter 1:  context-fid = 0.22513680194428304 

Iter 2:  context-fid = 0.23409674147223938 

Iter 3:  context-fid = 0.22452202772224578 

Iter 4:  context-fid = 0.23778806176274878 

Final Score:  0.23063392943106584 ± 0.007125535869128452


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [3]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [4]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)
min_len = min(x_real.shape[0], x_fake.shape[0])
x_real = x_real[:min_len]
x_fake = x_fake[:min_len]
assert x_real.shape[2] == x_fake.shape[2], "Real and fake data must have the same shape except for the batch dimension."
correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 6.581570094696123 

Iter 1:  cross-correlation = 7.710484651399119 

Iter 2:  cross-correlation = 6.320512339811292 

Iter 3:  cross-correlation = 6.314718166905891 

Iter 4:  cross-correlation = 8.424714120411076 

Final Score:  7.0703998746447 ± 1.1805959625230986


In [25]:
print((torch.std(x_real, dim=(1, 2)) == 0).sum())  # Count of zero-variance samples


tensor(0)
